# Starting a New GammaLike Analysis

The first step in a GammaLike analysis is to choose the spatial binning, energy binning, event selections etc.  GammaLike is especially computationally efficient because it first spends time generating the needed output files from the Fermi Science Tools.  After that, the instrument response functions are known and we can easily perform our template analyses on any models.  We will start by reproducing the analysis settings from [Calore et al (2015)](http://arxiv.org/abs/1409.0042), but with updates to Pass 8 data and the 3FGL point source catalog.  The comments below follow the Fermi Science Tools standard parameter nomencalture.  Comments inline describe the parameters, but additional information specific to Fermi can be found at http://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/.  

In addition to the analysis settings, we need to point GammaLike to the filepaths of the fermi event files and spacecraft files. 

In [1]:
import sys
sys.path.append('/data/GammaLike/')


# First we must import the Analyis 
from GammaLike import Analysis

#-------------------------------
# General Analysis Settings

# basepath: This defines the base directory for storing all the generated analysis files. 
# It should have at least 10-20 GB free. 
basepath = '/data/GammaLike/testing'

# tag: All the generated files carry this tag. Should be descriptive of analysis.
tag = 'P8R2_CLEAN_V6_calore' 

#--------------------------------
# Energy Binning Settings
# The energy binning by default matches that of Calore et al 2014 (see 1409.0042 Eq.2 and discussion)
# This consists of a few linear spaced bins at low energies followed by a rescursive binning definition.
# If custom binning is required, just set the bin edges using prefix_bins=... and change n_bins=0
prefix_bins=[300, 350, 400, 450, 500] # Fixed bins go here and are prepended to any additional recursive bins
# These 4 settings are for recursive bin definitions.  To disable set n_bins=0
n_bins = 20 # number of recursive bins
E_min = 5e2 # Starting recusrion energy
E_max = 5e5 # Max recursion energy
gamma = 1.45 # Recursion index. 

#--------------------------------
# Spatial Binning
healpix_order = 8 # Healpix order.  8 gives ~0.24 degree pixel edge width.  Increase of 1 halves the bin size.

#--------------------------------
# Fermitools settings

# phfile input to gtselect. Can be merged photon file or a text file containing paths of each weekly file
#phfile_raw = '/data/fermi_data_1-8-14/phfile.txt' 
phfile_raw = '/data/fermi_data_6-26-15/phfile.txt' 


# scfile [filename]
#           Spacecraft data file containing information such as the spacecraft
#           pointing as a function of time. This file could be generated by
#           gtorbsim for simulated observations (see the gtorbsim help for further
#           explanation) or it can be obtained from the FERMI
#           Science Support Center (FSSC) website for real observations.
scfile = '/data/fermi_data_6-26-15/lat_spacecraft_merged.fits' # Path to spacecraft file


# (evclass) [integer] 
#          Event class selection for pass 8 reprocessed data.
#       -----------------------------------------
#       |Class (Pass 8)|  evclass
#       |----------------------------------------
#       |Source        |  128  |
#       |Clean         |  256  |
#       |UltraCleanVeto|  1024 |
#       -----------------------------------------
# more info at http://fermi.gsfc.nasa.gov/ssc/data/analysis/documentation/Cicerone/Cicerone_LAT_IRFs/IRF_overview.html


evclass = 128 # FermiTools evclass
evtype = 3  


# filter [string]
#           This is the filter expression. The cuts are make using C-style
#           relational syntax like for example: ! (not), && (and), || (or), !=
#           (different), >, <, >=, <=, abs(), cos(), sin(), etc. Default is 
#           "DATA_QUAL>0 && LAT_CONFIG==1 && ABS(ROCK_ANGLE)<52".

gtfilter = 'DATA_QUAL>0 && LAT_CONFIG==1 && ABS(ROCK_ANGLE)<52' # gtmktime cut

# The fermi instrument response function.  Can get full list via command 'gtirfs'
# ******MAKE SURE THIS MATCHES evclass and evtype above!************
# Some common ones....
# P8R2_CLEAN_V6
# P8R2_CLEAN_V6::BACK
# P8R2_CLEAN_V6::EDISP0
# P8R2_CLEAN_V6::EDISP1
# P8R2_CLEAN_V6::EDISP2
# P8R2_CLEAN_V6::EDISP3
# P8R2_CLEAN_V6::FRONT
# P8R2_CLEAN_V6::PSF0
# P8R2_CLEAN_V6::PSF1
# P8R2_CLEAN_V6::PSF2
# P8R2_CLEAN_V6::PSF3
irf = 'P8R2_CLEAN_V6' 


# zmax [double]
#          Maximum apparent zenith angle (degrees). It ranges from 0 to 180 (default).
zmax = 90  # Max zenith angle cut.  90 is the standard cut for Pass8 data. 

#-----------------------------------------------------------------------------------------
# Some file paths containing fermi 3FGL catalog and the extended source templates

# path to 3FGL file (or 2FGL)
# can download this at http://fermi.gsfc.nasa.gov/ssc/data/access/lat/4yr_catalog/gll_psc_v14.fit
#      (2FGL is at at: http://fermi.gsfc.nasa.gov/ssc/data/access/lat/2yr_catalog/gll_psc_v08.fit)
# The extended source templates can be found here: http://fermi.gsfc.nasa.gov/ssc/data/access/lat/4yr_catalog/LAT_extended_sources_v15.tgz
fglpath = '/data/gll_psc_v16.fit' # 
#fglpath = '/data/gll_psc_v08.fit' # 2FGL
templateDir= '/data/Extended_archive_v15/Templates/'


# Path to a fermi diffuse model.  Used only to calculate point source masking per Calore et al.
# Can get this at http://fermi.gsfc.nasa.gov/ssc/data/access/lat/BackgroundModels.html
path_to_diffuse_model = '/data/fermi_background_models/gll_iem_v06.fits'



#---------------------------------------------------------
# Run this thing....
A = Analysis.Analysis(tag, basepath, 
                      phfile_raw, scfile, fglpath, templateDir, 
                      2**healpix_order, 
                      E_min, E_max, gamma, n_bins, prefix_bins,
                      evclass,  zmax, irf, evtype, gtfilter)



We have not yet generated the Fermitools exposure files so we can ignore the warning for now. 

# Loading the Analysis

The result above is a GammaLiek analysis file saved under "\$basepath/\$tag.GLanalysis".  This contains all of the analyis settings that will be needed in the future so we will be able to quickly load an analysis and get to work.  To load our analysis, we can use the following (replace the path of course):

In [5]:
basepath = '/data/GammaLike/testing/'
tag = 'P8R2_CLEAN_V6_calore' 
A = Analysis.Load(basepath+tag+'.GLanalysis')

# Generating the required Fermi Science Tools output

Now we need to generate the requisite exposure maps, PSF files, and photon data.  This is automated in GammaLike, but you must have the FermiScience tools in your system path. This is usually easily accomplished by adding the following lines to your ~/.bashrc file, (replace the paths!).

export FERMI_DIR="/data/fermitools/ScienceTools-v10r0p5-fssc-20150518-x86_64-unknown-linux-gnu-libc2.19-0/x86_64-unknown-linux-gnu-libc2.19-0"

source $FERMI_DIR/fermi-init.sh



This process will usually take a few hours because the Fermi Science Tools are very slow, and the datasets are big.  Luckily, it is a one time affair.   


In [11]:
# This will generate a shell script in the base directory specified above and execute it.  
# This calls each of the FermiTools executables individually.  Ocassionally you may run into problems with
# parameter values.  In this case, it can be helpful to exectute the scipt manually to debug.  

A.GenFermiData(runscript=True)

# Now we use the Fermi output to precompute some quantities for speed later.
A.GenExposureMap()
A.BinPhotons(outfile='binned_photons_'+A.tag+'.npy') # This is a healpix map of photon counts (for each energy+spatial bin)


Run this script to generate the required fermitools files for this analysis.
The script can be found at /data/GammaLike/testing/GenFermiData_P8R2_CLEAN_V6_calore_.sh
Get a beer. This will take a while...
running gtselect
Done.
running gtmktime
running gtltcube
running gtbindef
This is gtbindef version ScienceTools-v10r0p5-fssc-20150518
running gtpsf
Using evtype=3 (i.e., FRONT/BACK irfs)
running gtbin
This is gtbin version ScienceTools-v10r0p5-fssc-20150518
running gtexpcube2
Using evtype=3 (i.e., FRONT/BACK irfs)

Working on file /data/fermi_data_6-26-15/lat_spacecraft_merged.fits
.....................!
Computing binned exposure map....................!

Generating exposure map 100.00 %


# Generating Point Source Templates

In [12]:

#------------------------------------------------------------------------
# First we generate extended source templates.  PopulateROI will find all 
# point sources in the specified catalog and add their template to the 
# analysis template set.  For now we want them to be fixed so we will 
# merge them into a single template below.  This will make more sense 
# in later tutorials. 
A.PopulateROI([0,0], radius=360, fix_radius=360, include_point=False)

# Merge all sources into one template. 
master = np.zeros(A.templateList['LMC'].healpixCube.shape)
for key, t in A.templateList.items():
    master += t.healpixCube.toarray()

# Generate the point sources over the whole sky. This will also take a while...
# Still some problems with point sources near the poles so just go to 85 degrees until fixed...
A.GenPointSourceTemplate(pscmap=(A.basepath + '/PSC_' + A.tag + '_fgl3.npy'), l_range=(-180, 180), b_range=(-85, 85))

# Add this point source template to the extended source template and resave it. 
master += np.load(A.basepath + '/PSC_' + A.tag + '_fgl3.npy')
np.save(A.basepath + '/PSC_' + A.tag + '_fgl3_with_ext.npy', master)


Populating ROI with point sources: 123 of 3034
Populating ROI with point sources: 603 of 3034
Populating ROI with point sources: 640 of 3034
Populating ROI with point sources: 697 of 3034
Populating ROI with point sources: 930 of 3034
Populating ROI with point sources: 956 of 3034
Populating ROI with point sources: 1001 of 3034
Populating ROI with point sources: 1568 of 3034
Populating ROI with point sources: 1624 of 3034
Populating ROI with point sources: 1848 of 3034
Populating ROI with point sources: 1963 of 3034
Populating ROI with point sources: 1966 of 3034
Populating ROI with point sources: 2020 of 3034
Populating ROI with point sources: 2120 of 3034
Populating ROI with point sources: 2275 of 3034
Populating ROI with point sources: 2287 of 3034
Populating ROI with point sources: 2355 of 3034
Populating ROI with point sources: 2393 of 3034
Populating ROI with point sources: 2406 of 3034
Populating ROI with point sources: 2441 of 3034
Populating ROI with point sources: 2500 of 303

At this point, we have generated all of the files we need to start analyzing data. In the next tutorial we will set up an analysis over the full sky, and output a global diffuse model. 